# 16. 외부 라이브러리를 블로그에 활용
## 폼 모양 꾸미기
CreatView와 UpdateView를 사용해 만든 페이지는 아직 예쁘다고 보긴힘들다. django-crispy-forms를 이용하자.
### django-crispy-forms 적용하기
#### (1) django-crispy-forms 설치
pip으로 django-crispy-forms를 설치한다. 그리고 settings.py로들어가 crispy_forms를 추가하고 스타일은 bootstrap4로 지정해준다.

#### (2) post_form.html을 수정해 crispy_forms를 적용하기
post_form.html 제일 위에 {% load crispy_forms_tags %}를 추가하여 crispy_forms를 적용할 수 있게 하자. {{form}}에도 | crispy를 추가하면 더이상 폼이 테이블 형태로 나타나지 않으므로 table태그는 삭제하고 input태그도 crispy_form의 기본양식이 적용될 수 있게 하고 submit버튼과의 거리를 벌리기 위해 br태그도 주가해준다.

In [1]:
{% extends 'blog/base_full_width.html' %}
{% load crispy_forms_tags %}
{%  block head_title %}Create Post - Blog{% endblock %}

{% block main_area %}
    <h1>Create New Post</h1>
    <hr/>
    <form method="post", enctype="multiplart/form-data">{% csrf_token %}        
        {{form|crispy}}
        <div id="div_id_tags_str">
            <label for="id_tags_str">Tags : </label>
            <input type="text" name="tags_str" id="id_tags_str" class="textinputtextINput form-control"></td>
        </div>
        <br/>
        <button type="submit" class="btn btn-dark float-end">Submit</button>
    </form>
{% endblock %}  

SyntaxError: invalid syntax (<ipython-input-1-79bba4ede0fd>, line 1)

그럼 포스트 작성페지이가 훨씬 보기 좋아진 것을 알 수 있다. 마찬가지로 post_update_form,html도 수정해준다.
___
## 마크다운 적용하기
현재 만든 웹 사이트는 어떤 페이지에서도 포스트를 작성하더라도 줄바꿈이 적용되지 않는다. 글자 크기도 바꿀 수 없고 내용중간에 그림도 넣을 수 없어 가독성이 떨어진다.  
django-makrdownx를 설치하면 마크다운 문법을 적용할 수 있다.  
### django-markdownx로 마크다운문법 적용하기
#### (1) django-markdownx 적용하기
pip으로 django-markdownx를 설치해준다. 마찬자기로 settings.py를 열어서 INSTALLED_APPS에 mardownx를 추가해준다.  
urls.py에도 경로를 추가해주어야 원할하게 작동된다.(path('markdownx/',include('markdownx.urls')) 그리고 Post 모델의 content를 Textfield에서 MarkdownxField로 바꿔준다. 이제 modlels.py를 수정했으니 마이그레이션 해준다.  
post_form.html 과 post_update_form.html의 마지막 줄에 {{form.media}}을 추가해준다. 이제 작성페이지에서 글을 써보면 마크다운이 적용되어 렌더링 된 모습이 보인다.
#### (2) 작성한 내용에 마크다운 보이게 get_content_markdown() 메서드 만들기
하지만 submit 버튼을 클릭해보면 실제 글에는 마크다운이 적용되지않았다. 따라서 화면에 포스트를 렌더링 할때는 마크다운 문법으로 작성된 content 필드 값을 HTML로 변환하는 작업이 필요하다. 이 기능 역시 markdownx에서 제공한다. get_content_markdown()을 아래 방식으로 만든다. 이 메스드는 Post레코드의 content필드에 저장되어 있는 텍스트를 마크다운 문법을 적용해 HTML로 만든다.

In [2]:
def get_content_markdown(self):
    return markdown(self.content)

post_detail.html에 content를 그대로 가져오던 무분을 다음과 같이 get_content_markdown() 메서드를 한번 거쳐서 오도록 수정한다. 이렇게 하면 content필드에 마크다운 문법에 따라 저장된 text를 HTML로 변환 후 가져온다 이때 |safe를 같이 입력해 HTML 이스케이핑을 방지하는 필터도 추가한다.
> HTML 이스케이이란? :  
마크다운 문법으로 작성한 텍스트를 HTML로 변환해 가져올 때 왜 이스케이핑 방지 필터를 추가할까 예를들어 꺽쇠괄호 '<>' 를 사용한 텍스트를 div태그안에추가하면 HTML 문법 기준으로는 렌더링하기 모호하다. 꺽쇠기호는 HTML 태그의 시작을 알리는 기호이기 때문이다. 그래서 꺽쇠기호를 &lt, &gt같은 식으로 변환하는데 이런 행위를 HTML 이스케이핑이라고 한다. 그리고 변환한 요소를 엔티티(entities)라고도 한다.  
장고는 {{post.content}}와 같은 방식으로 불러온 값을 템플릿에 렌더링할때 자동으로 HTML 이스케이핑을 한다. 하지만 우리목적은 마크다운문법으로 작성한 내용을 HTML로 바꿔 그대로 화면에 출력하는것이기 때문에 |safe로 이스케이핑을 방지하는 것이다.  

이제 상세페이지를 열어보면 원하는 대로 포스트 내용이 나타난다.
#### (3) post_list.html 수정해 마크다운 적용
이제 포스트 목록 페이지도 수정해주자. 기존에 p.content로 되어있던 부분을 p.get_content_markdown으로 수정하자 그리고 이전과 달리 정보가 HTML로 넘어오기 떄문에  
truncateworkds를 truncateworkds_html로 수정한다. 마지막으로 |safe필터를 추가하자.  

관리자 페이지도 마크ㅏ운을 활용할 수 있게 수정하자. MarkdownxModel Admin을 임포터하고 수정하자. 그럼 관리자 페이지에서도 마크다운을 적용할 수 있다.
